In [1]:
#Pkg.add("Optim")
#Pkg.add("PyPlot")

using Pkg
using Optim
using PyPlot
using LinearAlgebra

┌ Info: Recompiling stale cache file /home/erichschulman/.julia/compiled/v1.0/Optim/R5uoh.ji for Optim [429524aa-4258-5aef-a3af-852621145aeb]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file /home/erichschulman/.julia/compiled/v1.0/PyPlot/oatAj.ji for PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1190


In [2]:
f(x) = (x[1]-5)^2

res = optimize(f,[1.],Newton())
fieldnames(typeof(res))

(:method, :initial_x, :minimizer, :minimum, :iterations, :iteration_converged, :x_converged, :x_tol, :x_abschange, :f_converged, :f_tol, :f_abschange, :g_converged, :g_tol, :g_residual, :f_increased, :trace, :f_calls, :g_calls, :h_calls)

In [3]:
X = [1, 2, 3, 4, 5]
L = 3

function moving_average(x,l)
    N= length(x)
    h = zeros(N, N - l + 1)
    for i =1:(N-l+1)
        println(i)
        h[i:i+l-1,i] = 1/l*ones(l)
    end
    return h
end


moving_average (generic function with 1 method)

In [30]:
function no_win(p,g)
    l = length(p)
    gs = reverse(g(p))
    result = ones(l)
    for i = 2:l
        for j=i:l
            result[i-1] = result[i-1]*gs[j]
        end
    end
    return reverse(result)
end


function create_obj(u,g)
    obj(p) = -1*u(p,g)
    return obj
end


f1(p) = clamp!(p,0, 1)
u1(p,g) = sum(p.*(ones(size(p)) - g(p)).*no_win(p,g))

P = [.5,.5,.5]
res = optimize(create_obj(u1,f1),[.5,.5,.5])

println(res.minimizer)
println(no_win(res.minimizer,f1))
println(f1(res.minimizer))

[0.695226, 0.625048, 0.499877]
[1.0, 0.695226, 0.43455]
[0.695226, 0.625048, 0.499877]


In [43]:
function update_beliefs(p,g)
    l = length(p)
    gs = g(p)
    result = gs
    for i = 2:l
        result[i] = Int( p[i] <= minimum(p[1:i-1]) )*result[i]/gs[i-1]
    end
    return result
end

P = [.7, .6, .5]
u2(p,g) = p .* update_beliefs(p,g)
obj2(p) = -1*u2(p,f1)
println("first try ", obj2(P))


first try [-0.49, -0.734694, -0.340278]


In [44]:
res2 = optimize(obj2,[.5,.4,.3])
println(res2.minimizer)
print(obj2(res2.minimizer))
print(f1(res2.minimizer))

MethodError: MethodError: Cannot `convert` an object of type Array{Float64,1} to an object of type Float64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...